# Liquidity Exit Agent

The ``TVLBasedLiquidityExitAgent`` monitors a liquidity pool's Total Value Locked (TVL) and automatically withdraws a user’s position when TVL falls below a specified threshold, mitigating risks like slippage or impermanent loss. It integrates DeFiPy’s RemoveLiquidity for exits and Web3Scout’s event feeds for real-time TVL updates, supporting off-chain testing and on-chain execution.

* To download notebook to this tutorial, see [here](https://github.com/defipy-devs/defipy-docs/blob/main/docs/onchain/onchain/price_threshold_swap.ipynb) 

### 📘 Notable Classes

---

* **Class**: 📘 ``defipy.agents.config.TVLExitConfig`` 
    * **Purpose**: Configuration for price threshold swap agent.
        * **Parameters**:
            * `threshold`: Price threshold to check (`float`).
            * `pool_address`: UniV2 pool address (`str`).
            * `provider_url`: Provider URL (eg, infura.io)(`str`).
            * `platform`: Platform where pool resides (eg, uniswap) (`str`).
            * `abi_name`: ABI handle pointing to pool contract in JSON (eg, UniswapV2Pair) (`str`).

---
         
* **Class**: 📘 ``defipy.agents.TVLBasedLiquidityExitAgent`` 
    * **Purpose**: Reactive DeFi agent for determining price threshold.
        * **Parameters**:
            * `config`: Agent configuration parameters (`PriceThresholdConfig`).
    * **Methods**:
        * ``apply()``
            * **Purpose**: Apply price threshold agent
        * ``run_batch(tkn: ERC20, events: dict)``
            * **Purpose**: Run AI price agent on batch data 
            * **Parameters**:
                * `tkn`: Swap token (`ERC20`).
                * `events`: Dictionary of sync events (`dict`).
        * ``prime_pool_state(start_block: int, user_nm: str)``
            * **Purpose**: Initialize off-chain pool using information in start_block
            * **Parameters**:
                * `start_block`: Start block (`int`).
                * `user_nm`: Account name of mock off-chain pool (`str`). 
        * ``get_token_price(tkn1_over_tkn0: bool, block_num: int)``
            * **Purpose**: Get current price of pool  
            * **Parameters**:
                * `tkn1_over_tkn0`: price = tkn1/tkn0 (true) (`bool`).
                * `block_num`: block number (`int`).
        * ``check_condition(threshold: float, tkn1_over_tkn0: bool)``
            * **Purpose**: Calculate initial token value of position   
            * **Parameters**:
                * `threshold`: override config price threshold (optional) (`float`).
                * `tkn1_over_tkn0`: price = tkn1/tkn0 (true, optional) (`float`).
        * ``get_w3()``
            * **Purpose**: Getter function for web3.py connector object
        * ``get_abi()``
            * **Purpose**: Getter function for ABI json data
        * ``get_contract_instance()``
            * **Purpose**: Getter function for pool contract instance
        * ``get_lp_data()``
            * **Purpose**: Getter function for pool data associated with `pool_address` from config


In [1]:
from defipy import *
from web3scout import *

### Setup Agent

In [2]:
tvl_threshold = 10000000
exit_percentage = 1.0
pair_address = "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc"
provider_url = "https://mainnet.infura.io/v3/9624e3e5c40f4ac3958b79fa5aa2562d"
platform = Platform.AGNOSTIC
abi_name = JSONContract.UniswapV2Pair
user_position = 10

config = TVLExitConfig(
    tvl_threshold = tvl_threshold,
    exit_percentage = exit_percentage,
    pool_address = pair_address,
    provider_url = provider_url,
    platform = platform,
    abi_name = abi_name,
    user_position = user_position
)

agent = TVLBasedLiquidityExitAgent(config)
agent.init()

print(f"Monitoring TVL changes @ pool address {pair_address}")

Monitoring TVL changes @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc


In [3]:
abi = ABILoad(platform, abi_name)
connect = ConnectW3(provider_url)
connect.apply()

last_block = connect.get_w3().eth.block_number
start_block = last_block - 250

# Grab batch sync events from pool
rEvents = RetrieveEvents(connect, abi)
events = rEvents.apply(EventType.SYNC, address = pair_address, start_block=start_block, end_block=last_block)
rEvents.to_dataframe(events).head(2)

,blockNumber,event,address,blockHash,logIndex,transactionHash,transactionIndex,args
0,23042432,Sync,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0x268e248d6d7d17363a75ebe59d92818e16c2bc33947c...,767,0xd661224622c13d770730d3a5c8e8cb204e2eaa619a40...,18,"{'reserve0': 14595022995236, 'reserve1': 39504..."
1,23042437,Sync,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0x997dd29630329b4fc93daef3ed96f50a7318b9804db4...,247,0x3887d48b6b8614318827c9de49d2b68c68f5883e8fbe...,67,"{'reserve0': 14595031353137, 'reserve1': 39504..."


In [4]:
tkn0 = agent.get_lp_data().tkn0
tkn1 = agent.get_lp_data().tkn1

lp = agent.prime_mock_pool(start_block, 'user')
lp.summary()

Exchange USDC-WETH (LP)
Reserves: USDC = 14594840.921519, WETH = 3950.485213265235
Liquidity: 0.08984440448421524 



In [5]:
lp = agent.update_mock_pool(lp, last_block)
lp.summary()

Exchange USDC-WETH (LP)
Reserves: USDC = 14582125.977117, WETH = 3954.0127778609135
Liquidity: 0.08984440448421524 



In [6]:
mock_lp_pos = agent.take_mock_position(lp, tkn0, 'user', 10)

In [11]:
# agent.get_pool_tvl(lp, tkn0, last_block)
# agent.check_condition(lp, tkn0, tvl_threshold, last_block)
# agent.apply(lp, tkn0, 'user', last_block)

In [10]:
agent.run_batch(lp, tkn0, 'user', events)

Block 23042432: TVL threshold condition met for USDC-WETH LP
Block 23042437: TVL threshold condition met for USDC-WETH LP
Block 23042462: TVL threshold condition met for USDC-WETH LP
Block 23042470: TVL threshold condition met for USDC-WETH LP
Block 23042487: TVL threshold condition met for USDC-WETH LP
Block 23042491: TVL threshold condition met for USDC-WETH LP
Block 23042492: TVL threshold condition met for USDC-WETH LP
Block 23042496: TVL threshold condition met for USDC-WETH LP
Block 23042499: TVL threshold condition met for USDC-WETH LP
Block 23042502: TVL threshold condition met for USDC-WETH LP
Block 23042503: TVL threshold condition met for USDC-WETH LP
Block 23042507: TVL threshold condition met for USDC-WETH LP
Block 23042507: TVL threshold condition met for USDC-WETH LP
Block 23042508: TVL threshold condition met for USDC-WETH LP
Block 23042511: TVL threshold condition met for USDC-WETH LP
Block 23042512: TVL threshold condition met for USDC-WETH LP
Block 23042519: TVL thre